<a href="https://colab.research.google.com/github/YellaCRE/SportRecog/blob/main/%EB%AC%B8%ED%99%94_%EC%B6%94%EC%B2%9C%EC%8B%9C%EC%8A%A4%ED%85%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#결측치 확인

In [ ]:
def check_missing_col(dataframe):
    missing_col = []
    counted_missing_col = 0
    for i, col in enumerate(dataframe.columns):
        missing_values = sum(dataframe[col].isna())
        is_missing = True if missing_values >= 1 else False
        if is_missing:
            counted_missing_col += 1
            print(f'결측치가 있는 컬럼은: {col}입니다')
            print(f'해당 컬럼에 총 {missing_values}개의 결측치가 존재합니다.')
            missing_col.append([col, dataframe[col].dtype])
    if counted_missing_col == 0:
        print('결측치가 존재하지 않습니다')
    return missing_col

In [ ]:
# 결측치를 처리하는 함수를 작성
def handle_na(data, missing_col):
    temp = data.copy()
    for col, dtype in missing_col:
        if dtype == 'O':
            # 범주형 feature가 결측치인 경우 해당 행들을 삭제
            temp = temp.dropna(subset=[col])
    return temp

#데이터 불러오기

In [ ]:
import os
import pandas as pd
pd.set_option('mode.chained_assignment',  None) #수정오류 무시
import numpy as np
from math import sqrt
from tqdm import tqdm_notebook as tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

In [ ]:
path = "/content/drive/MyDrive/2022 휴학/문화관광공모전/data/"

In [ ]:
#csv 형식으로 된 데이터 파일을 읽어옵니다. 
data = pd.read_csv(path + "person_data_clu.csv")

#EDA

In [ ]:
data = data[data['year']==2021]

ID

In [ ]:
df_ID = data[['ID','year']]
#df_ID

Best

In [ ]:
#가장 만족하는 여가활동 7점 부여
df_best = data[['Q4_1','Q4_2','Q4_3']]
df_best.insert(3,'rate_3',[7]*len(df_best))
df_best.insert(2,'rate_2',[7]*len(df_best))
df_best.insert(1,'rate_1',[7]*len(df_best))
#df_best

지난 1년간 가장 많이 한 여가활동

In [ ]:
df_past = data[['Q2_1_1','Q2_6_1','Q2_1_2','Q2_6_2','Q2_1_3','Q2_6_3','Q2_1_4','Q2_6_4','Q2_1_5','Q2_6_5']]
df_past.replace('7 매우 만족한다',7,inplace=True)
df_past.replace('1 매우 불만족 한다',1,inplace=True)
df_past.replace('4 보통이다',4,inplace=True)
#df_past

평일 주말에 희망하는 여가활동

In [ ]:
#가장 희망하는 여가활동 6점 부여
df_week = data[['Q11_1_C','Q11_2_C','Q11_1_D','Q11_2_D','Q11_3_C','Q11_3_D']]
df_week.insert(6,'rate_s2',[6]*len(df_week))
df_week.insert(5,'rate_s1',[6]*len(df_week))
#df_week

In [ ]:
df_wknd = data[['Q12_1_C','Q12_2_C','Q12_1_D','Q12_2_D','Q12_3_C','Q12_3_D']]
df_wknd.insert(6,'rate_s4',[6]*len(df_week))
df_wknd.insert(5,'rate_s3',[6]*len(df_week))
#df_wknd

In [ ]:
#평점 데이터로 전처리
def makerate(n):
  temp = pd.concat([df_best,df_past,df_week,df_wknd],axis=1)
  temp_rate = pd.concat([df_ID,temp.iloc[:,[2*n-2,2*n-1]]],axis=1)
  temp_rate.columns = ['ID','year','leisureID','rating']
  return temp_rate

In [ ]:
df_rating = pd.concat([makerate(1),makerate(2),makerate(3),makerate(4),makerate(5),makerate(6),makerate(7),makerate(8),makerate(9),makerate(10),makerate(11),makerate(12),makerate(13),makerate(14),makerate(15)])

In [ ]:
df_rating

,ID,year,leisureID,rating
30646,202115021,2021,취미생활중,7
30647,202115022,2021,취미생활중,7
30648,202115023,2021,취미생활여,7
30649,202115024,2021,취미생활중,7
30650,202115025,2021,취미생활중,7
...,...,...,...,...
40690,202125065,2021,없음,6
40691,202125066,2021,"스포츠 경기 직접관람- 경기장방문관람(축구, 야구,농구, 배구 등)",6
40692,202125067,2021,"스포츠 경기 직접관람- 경기장방문관람(축구, 야구,농구, 배구 등)",6
40693,202125068,2021,"스포츠 경기 직접관람- 경기장방문관람(축구, 야구,농구, 배구 등)",6


#전처리

In [ ]:
#없음 데이터 제거
df_rating=df_rating[df_rating['leisureID']!='없음']

In [ ]:
df_rating.dropna(subset=['leisureID'],inplace=True)
df_rating.dropna(subset=['rating'],inplace=True)

In [ ]:
missing_col = check_missing_col(df_rating)

결측치가 존재하지 않습니다


데이터 형식 변환

In [ ]:
df_rating = df_rating.astype({'rating':'int'})

In [ ]:
#불필요한 컬럼, 중복데이터 제거
df_rating.drop(['year'],axis=1,inplace=True)
df_rating.drop_duplicates(['ID','leisureID'],inplace=True)

결측치,중복값 처리

In [ ]:
df_rating.dropna(subset=['leisureID'],inplace=True)

In [ ]:
# 결측치 처리가 잘 되었는지 확인
missing_col = check_missing_col(df_rating)

결측치가 존재하지 않습니다


In [ ]:
df_rating

,ID,leisureID,rating
30646,202115021,취미생활중,7
30647,202115022,취미생활중,7
30648,202115023,취미생활여,7
30649,202115024,취미생활중,7
30650,202115025,취미생활중,7
...,...,...,...
40637,202125012,"스포츠 경기 직접관람- 경기장방문관람(축구, 야구,농구, 배구 등)",6
40640,202125015,"스포츠 경기 직접관람- 경기장방문관람(축구, 야구,농구, 배구 등)",6
40642,202125017,"스포츠 경기 직접관람- 경기장방문관람(축구, 야구,농구, 배구 등)",6
40686,202125061,"스포츠 경기 직접관람- 경기장방문관람(축구, 야구,농구, 배구 등)",6


#모델테스트

In [ ]:
train_df, test_df = train_test_split(df_rating, test_size=0.3, random_state=1004)

Create Sparse Matrix

In [ ]:
sparse_matrix = train_df.groupby('leisureID').apply(lambda x: pd.Series(x['rating'].values, index=x['ID'])).unstack()
sparse_matrix.index.name = 'leisureID'

sparse_matrix

ID,202115021,202115022,202115023,202115024,202115025,202115026,202115027,202115028,202115029,202115030,...,202125060,202125061,202125062,202125063,202125064,202125065,202125066,202125067,202125068,202125069
leisureID,,,,,,,,,,,,,,,,,,,,,
"격투 스포츠 경기관람(태권도, 유도, 합기도, 검도,권투 등)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"격투 스포츠(태권도, 유도, 합기도, 검도, 권투 등)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
골프,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN
기타,4.0,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,...,6.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN
"농구, 배구, 야구, 축구, 족구",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.0,6.0,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN
당구/포켓볼,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"댄스스포츠(탱고, 왈츠, 자이브, 맘보,폴카, 차차차등)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
등산,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"배드민턴/줄넘기/맨손, 스트레칭 체조/훌라후프",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


코사인 유사도

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def cossim_matrix(a, b):
    cossim_values = cosine_similarity(a.values, b.values)
    cossim_df = pd.DataFrame(data=cossim_values, columns = a.index.values, index=a.index)

    return cossim_df

유저기반

In [ ]:
user_sparse_matrix = sparse_matrix.fillna(3).transpose()
user_cossim_df = cossim_matrix(user_sparse_matrix, user_sparse_matrix)

sportId_grouped = train_df.groupby('leisureID')
user_prediction_result_df = pd.DataFrame(index=list(sportId_grouped.indices.keys()), columns=user_sparse_matrix.index)

In [ ]:
for sportId, group in tqdm(sportId_grouped):
    group['ID'].dropna()
    user_sim = user_cossim_df.loc[group['ID']]
    user_rating = group['rating']
    sim_sum = user_sim.sum(axis=0)

    pred_ratings = np.matmul(user_sim.T.to_numpy(), user_rating) / (sim_sum+1)
    user_prediction_result_df.loc[sportId] = pred_ratings

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


  0%|          | 0/31 [00:00<?, ?it/s]

In [ ]:
user_prediction_result_df.head(10)

ID,202115021,202115022,202115023,202115024,202115025,202115026,202115027,202115028,202115029,202115030,...,202125060,202125061,202125062,202125063,202125064,202125065,202125066,202125067,202125068,202125069
"격투 스포츠 경기관람(태권도, 유도, 합기도, 검도,권투 등)",5.897895,5.898333,5.897479,5.897729,5.899326,5.899054,5.898598,5.898193,5.897845,5.897511,...,5.896354,5.898416,5.896826,5.898146,5.896811,5.897804,5.896605,5.89827,5.897619,5.8973
"격투 스포츠(태권도, 유도, 합기도, 검도, 권투 등)",6.360615,6.363621,6.359113,6.361526,6.361058,6.357951,6.357763,6.362659,6.363284,6.360724,...,6.360109,6.358713,6.358037,6.358269,6.357826,6.358702,6.355212,6.354977,6.35574,6.358232
골프,6.361754,6.358579,6.361881,6.35791,6.36181,6.360801,6.36177,6.358657,6.358131,6.358661,...,6.359419,6.357933,6.361292,6.358776,6.3591,6.361205,6.365518,6.360842,6.360663,6.360624
기타,5.876157,5.874722,5.873187,5.887549,5.874864,5.872672,5.875619,5.874799,5.873401,5.874608,...,5.880365,5.871496,5.872352,5.873246,5.879622,5.874017,5.874401,5.874671,5.872498,5.871908
"농구, 배구, 야구, 축구, 족구",6.314411,6.314798,6.31418,6.314178,6.316611,6.316232,6.315441,6.314396,6.314875,6.313248,...,6.319597,6.318721,6.315049,6.313884,6.312859,6.316351,6.313701,6.321057,6.315668,6.315604
당구/포켓볼,6.077022,6.077168,6.077148,6.076495,6.078033,6.076724,6.078323,6.077333,6.076603,6.080492,...,6.075896,6.075782,6.075886,6.076833,6.075906,6.076407,6.076652,6.075813,6.075835,6.075504
"댄스스포츠(탱고, 왈츠, 자이브, 맘보,폴카, 차차차등)",6.064592,6.064812,6.063702,6.063339,6.065725,6.063775,6.064548,6.064765,6.064544,6.061755,...,6.062846,6.062683,6.063385,6.063144,6.06207,6.064623,6.061449,6.061859,6.062693,6.063739
등산,6.811081,6.811175,6.810978,6.810412,6.811128,6.811089,6.811231,6.811244,6.810829,6.810859,...,6.81008,6.810439,6.810825,6.81136,6.811363,6.81121,6.811238,6.811098,6.810918,6.811296
"배드민턴/줄넘기/맨손, 스트레칭 체조/훌라후프",5.828386,5.829291,5.828939,5.828633,5.830665,5.830277,5.831642,5.829557,5.828188,5.82942,...,5.827203,5.828901,5.828217,5.830089,5.829472,5.829289,5.830409,5.83164,5.82941,5.828511
"볼링, 탁구",6.153915,6.154616,6.158109,6.154267,6.155221,6.153065,6.155216,6.154752,6.154584,6.153973,...,6.154375,6.153065,6.15433,6.15289,6.152909,6.155413,6.152808,6.153716,6.153411,6.153826


In [ ]:
user_prediction_result_df = user_prediction_result_df.transpose()

평가

In [ ]:
def evaluate(test_df, prediction_result_df):
  groups_with_sport_ids = test_df.groupby(by='leisureID')
  groups_with_user_ids = test_df.groupby(by='ID')
  intersection_sport_ids = sorted(list(set(list(prediction_result_df.columns)).intersection(set(list(groups_with_sport_ids.indices.keys())))))
  intersection_user_ids = sorted(list(set(list(prediction_result_df.index)).intersection(set(groups_with_user_ids.indices.keys()))))

  print(len(intersection_sport_ids))
  print(len(intersection_user_ids))

  compressed_prediction_df = prediction_result_df.loc[intersection_user_ids][intersection_sport_ids]
  # compressed_prediction_df

  # test_df에 대해서 RMSE 계산
  grouped = test_df.groupby(by='ID')
  result_df = pd.DataFrame(columns=['rmse'])
  for userId, group in tqdm(grouped):
      if userId in intersection_user_ids:
          pred_ratings = compressed_prediction_df.loc[userId][compressed_prediction_df.loc[userId].index.intersection(list(group['leisureID'].values))]
          pred_ratings = pred_ratings.to_frame(name='rating').reset_index().rename(columns={'index':'leisureID','rating':'pred_rating'})
          actual_ratings = group[['rating', 'leisureID']].rename(columns={'rating':'actual_rating'})

          final_df = pd.merge(actual_ratings, pred_ratings, how='inner', on=['leisureID'])
          final_df = final_df.round(4) # 반올림

          # if not final_df.empty:
          #     rmse = sqrt(mean_squared_error(final_df['rating_actual'], final_df['rating_pred']))
          #     result_df.loc[userId] = rmse
          #     # print(userId, rmse)
    
  return final_df

유저기반

In [ ]:
result_df = evaluate(test_df, user_prediction_result_df)
print(result_df)
print(f"RMSE: {sqrt(mean_squared_error(result_df['actual_rating'].values, result_df['pred_rating'].values))}")

31
8112


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  app.launch_new_instance()


  0%|          | 0/8210 [00:00<?, ?it/s]

   actual_rating leisureID pred_rating
0              6        수영    6.144331
1              7        여행     6.81605
RMSE: 0.16533134765830862


#모델

In [ ]:
sparse_matrix = df_rating.groupby('leisureID').apply(lambda x: pd.Series(x['rating'].values, index=x['ID'])).unstack()
sparse_matrix.index.name = 'leisureID'

user_sparse_matrix = sparse_matrix.fillna(3).transpose()
user_cossim_df = cossim_matrix(user_sparse_matrix, user_sparse_matrix)

sportId_grouped = df_rating.groupby('leisureID')
user_prediction_result_df = pd.DataFrame(index=list(sportId_grouped.indices.keys()), columns=user_sparse_matrix.index)

for sportId, group in tqdm(sportId_grouped):
    group['ID'].dropna()
    user_sim = user_cossim_df.loc[group['ID']]
    user_rating = group['rating']
    sim_sum = user_sim.sum(axis=0)

    pred_ratings = np.matmul(user_sim.T.to_numpy(), user_rating) / (sim_sum+1)
    user_prediction_result_df.loc[sportId] = pred_ratings

user_prediction_result_df = user_prediction_result_df.transpose()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # Remove the CWD from sys.path while we load stuff.


  0%|          | 0/31 [00:00<?, ?it/s]

#필터링

In [ ]:
df = data

#사용자 정보
gender = 'M'
age = 20
marriage = '미혼'
worktime = 30
purpose = '즐거움'
clu = 2

성별


In [ ]:
df = df[df['DM1']==gender]

결혼

In [ ]:
df = df[df['DM5']==marriage]

근무시간

In [ ]:
df = df[(df['Q47_N']>=worktime-5)&(df['Q47_N']<=worktime+15)]

나이

In [ ]:
df = df[(df['DM2']==age)|(df['DM2']==age+10)|(df['DM2']==age-10)]

목적

In [ ]:
df = df[df['Q3']==purpose]

경험

In [ ]:
df = df[df['cluster']==clu]

In [ ]:
df

,Unnamed: 0,ID,year,DM1,DM2,DM11,Q45,Q47_N,DM5,Q3,...,Q11_2_D,Q11_3_C,Q11_3_D,Q12_1_C,Q12_2_C,Q12_1_D,Q12_2_D,Q12_3_C,Q12_3_D,cluster
30885,30885,202115260,2021,M,20,대전,하였다,40.0,미혼,즐거움,...,6.0,"스포츠 경기 간접관람- TV, DMB를 통한관람(축구, 야구, 농구, 배구 등)","볼링, 탁구",온라인게임 경기관람(e-스포츠 경기 포함),6.0,"농구, 배구, 야구, 축구, 족구",5.0,"스포츠 경기 간접관람- TV, DMB를 통한관람(축구, 야구, 농구, 배구 등)","볼링, 탁구",2
30919,30919,202115294,2021,M,20,부산,하였다,40.0,미혼,즐거움,...,5.0,없음,헬스(보디빌딩)/에어로빅,없음,NaN,헬스(보디빌딩)/에어로빅,5.0,"스포츠 경기 직접관람- 경기장방문관람(축구, 야구,농구, 배구 등)",헬스(보디빌딩)/에어로빅,2
30927,30927,202115302,2021,M,20,경남,하였다,30.0,미혼,즐거움,...,NaN,없음,없음,없음,NaN,없음,NaN,없음,없음,2
30969,30969,202115344,2021,M,30,부산,하였다,40.0,미혼,즐거움,...,NaN,없음,없음,없음,NaN,없음,NaN,없음,없음,2
31144,31144,202115519,2021,M,30,경남,하였다,45.0,미혼,즐거움,...,NaN,없음,헬스(보디빌딩)/에어로빅,없음,NaN,없음,NaN,없음,헬스(보디빌딩)/에어로빅,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40517,40517,202124892,2021,M,10,전남,하지 않았다,28.0,미혼,즐거움,...,NaN,"스포츠 경기 직접관람- 경기장방문관람(축구, 야구,농구, 배구 등)","농구, 배구, 야구, 축구, 족구",없음,NaN,없음,NaN,온라인게임 경기관람(e-스포츠 경기 포함),헬스(보디빌딩)/에어로빅,2
40531,40531,202124906,2021,M,10,전남,하지 않았다,28.0,미혼,즐거움,...,NaN,"스포츠 경기 직접관람- 경기장방문관람(축구, 야구,농구, 배구 등)","농구, 배구, 야구, 축구, 족구",없음,NaN,없음,NaN,"스포츠 경기 직접관람- 경기장방문관람(축구, 야구,농구, 배구 등)","농구, 배구, 야구, 축구, 족구",2
40556,40556,202124931,2021,M,20,광주,하였다,40.0,미혼,즐거움,...,NaN,"스포츠 경기 직접관람- 경기장방문관람(축구, 야구,농구, 배구 등)",요가/필라테스/태보,없음,NaN,수영,7.0,"스포츠 경기 직접관람- 경기장방문관람(축구, 야구,농구, 배구 등)",수영,2
40559,40559,202124934,2021,M,20,광주,하였다,40.0,미혼,즐거움,...,NaN,"스포츠 경기 직접관람- 경기장방문관람(축구, 야구,농구, 배구 등)",요가/필라테스/태보,없음,NaN,없음,NaN,"스포츠 경기 직접관람- 경기장방문관람(축구, 야구,농구, 배구 등)",요가/필라테스/태보,2


In [ ]:
ID_list=df['ID'].unique()

#결과출력

In [ ]:
#데이터가 있는 여가생활만 평가
temp_rating = df_rating[df_rating['ID'].isin(ID_list)]
temp_list = temp_rating['leisureID'].unique()

In [ ]:
#전체데이터의 평균점수, 목표군집의 평균점수 비교
test = user_prediction_result_df.loc[ID_list].mean()
aver = user_prediction_result_df.mean()
temp = test - aver
rec_list = temp.loc[temp_list].sort_values(ascending=False)

In [ ]:
rec_list.head(10)

취미생활남                     0.003049
예술관람                      0.001705
볼링, 탁구                    0.000742
승마, 암벽등반, 철인삼종경기, 서바이벌    0.000717
헬스(보디빌딩)/에어로빅             0.000621
취미생활중                     0.000560
스노보드, 스키 등                0.000338
사이클링/산악자전거                0.000316
수영                        0.000241
사교활동                      0.000229
dtype: float64

In [ ]:
temp_rating[temp_rating['leisureID']=='스노보드, 스키 등']

,ID,leisureID,rating
38760,202123135,"스노보드, 스키 등",7
39082,202123457,"스노보드, 스키 등",7
33591,202117966,"스노보드, 스키 등",6
37148,202121523,"스노보드, 스키 등",6


In [ ]:
temp_rating[temp_rating['leisureID']=='사이클링/산악자전거']

,ID,leisureID,rating
33242,202117617,사이클링/산악자전거,7
34976,202119351,사이클링/산악자전거,7
31262,202115637,사이클링/산악자전거,7
34819,202119194,사이클링/산악자전거,7
32901,202117276,사이클링/산악자전거,6
37770,202122145,사이클링/산악자전거,6
39888,202124263,사이클링/산악자전거,6
